# Exploratory and statistic indsights

In [37]:
# Load data and refine it a bit (string to datetime, merge both mechanical and electrical, check for null values, ...)

import pandas as pd
import numpy as np
import geopandas as gpd


column_names = ['date','capacity','available_mechanical','available_electrical','station_name','station_geo','operative']
df = pd.read_csv('data/historique_stations.csv', header=None, names=column_names)

df['available'] = df['available_mechanical'] + df['available_electrical']
df = df.drop(columns = ['available_mechanical', 'available_electrical'])
df['date'] = pd.to_datetime(df['date'])

print(df.isnull().values.any())
display(df)
df.dtypes

# Trim every string in dataframe because whitespace will become annoying after pivot

trim_strings = lambda x: x.strip() if isinstance(x, str) else x
df = df.applymap(trim_strings)

False


,date,capacity,station_name,station_geo,operative,available
0,2020-11-26 12:59:00+00:00,35,Benjamin Godard - Victor Hugo,"48.86598,2.27572",True,9
1,2020-11-26 12:59:00+00:00,55,André Mazet - Saint-André des Arts,"48.85376,2.33910",True,27
2,2020-11-26 12:59:00+00:00,20,Charonne - Robert et Sonia Delauney,"48.85591,2.39257",True,0
3,2020-11-26 12:59:00+00:00,21,Toudouze - Clauzel,"48.87930,2.33736",True,1
4,2020-11-26 12:59:00+00:00,30,Mairie du 12ème,"48.84086,2.38755",True,4
...,...,...,...,...,...,...
10986725,2021-04-09 14:37:00+00:00,38,Général Michel Bizot - Claude Decaen,"48.83481,2.40093",True,6
10986726,2021-04-09 14:37:00+00:00,20,Ivry - Baudricourt,"48.82470,2.36311",True,3
10986727,2021-04-09 14:37:00+00:00,39,Saint-Mandé - Docteur Arnold Netter,"48.84463,2.40495",True,17
10986728,2021-04-09 14:37:00+00:00,21,Saint-Marcel - Hôpital,"48.83950,2.36099",True,16


## How many stations, total capacity (the highest and the lowest)

In [38]:
num_stations = len(df['station_name'].unique())
highest_capacity = df.sort_values('capacity',ascending=False).groupby(by='station_name').first()['capacity'].sum()
lowest_capacity = df.sort_values('capacity').groupby(by='station_name').first()['capacity'].sum()

print('Number of stations :')
print(num_stations)

print('Highest total capacity :')
print(highest_capacity)

print('Lowest total capacity :')
print(lowest_capacity)


Number of stations :
1396
Highest total capacity :
43982
Lowest total capacity :
43737


## Should we ignore stations ? Check for unresponsive stations, capacity = 0...

### How many total dates are available ? Is is the case for every stations ?

In [39]:
date_arr1 = df['date'].unique()
num_date = len(date_arr1)
latest = date_arr1.max()

print("Number of different dates :")
print(num_date)
print("Latest date :")
print(latest)

Number of different dates :
7866
Latest date :
2021-04-09 14:37:00+00:00


### Check for unresponsive stations

#### Too few data

In [40]:
# Display station when more than half values are missing

stations_groupdf = df.groupby(by='station_name')

val_per_stations_df = stations_groupdf['date'].nunique().sort_values()
too_few_val_per_stations_df = val_per_stations_df[val_per_stations_df < num_date/2]
display(too_few_val_per_stations_df)

station_name
Jean Jaurès - Paul Lafargue        2
station formation alfortville      5
Parmentier - Abel rabaud         612
Arago - Paul Lafargue            680
Name: date, dtype: int64

#### Latest response too old

In [41]:
# Display station when delay is more than an hour

latest_df = latest - stations_groupdf['date'].max()

display(latest_df[latest_df > np.timedelta64(1, 'h')])

station_name
Parmentier - Abel rabaud        126 days 21:36:00
station formation alfortville    30 days 22:05:00
Name: date, dtype: timedelta64[ns]

In [42]:
# We will ignore the stations with too few data

df = df[~df['station_name'].isin(too_few_val_per_stations_df.index)]

### Capacity of 0 (probably bugs) : 

In [43]:
no_cap_df = df[df['capacity'] == 0].groupby('station_name')
display(no_cap_df.size().sort_values())

station_name
Hôpital Européen Georges Pompidou          2
Chabanais - Petits Champs               1154
Université Paris Dauphine               3884
Parc Floral du Bel Air                  3978
Berthier - Porte de Clichy              7866
Champs de Manoeuvre - Cartoucherie      7866
Longchamp - Suresnes                    7866
Quai de la Gare - Pont de Bercy         7866
Saint-Michel - Luxembourg               7866
Verdun - Pierre et Marie Curie          7866
Victor Massé - Jean-Baptiste Pigalle    7866
dtype: int64

### Capacity of 0 but with bikes available (definitely bugs) :

In [44]:
display(df[(df['capacity'] == 0) & (df['available'] > 0)].groupby('station_name').size().sort_values())

station_name
Chabanais - Petits Champs    31
dtype: int64

## Date continuity analysis 

In [45]:
# Can't .diff() a DatetimeArray so do it manually. Remove the first one (and last to get matching lengths), then substract.

date_arr2 = date_arr1[:-1]
date_arr1 = date_arr1[1:]
granularity = date_arr1 - date_arr2
print(sorted(granularity, reverse = True)[:5])
print(sorted(granularity)[:5])

#Time granularity is uneven, we should resample and pivot the table to have a timeseries dataframe

[Timedelta('3 days 19:46:00'), Timedelta('0 days 03:41:00'), Timedelta('0 days 03:24:00'), Timedelta('0 days 02:15:00'), Timedelta('0 days 02:05:00')]
[Timedelta('0 days 00:01:00'), Timedelta('0 days 00:01:00'), Timedelta('0 days 00:01:00'), Timedelta('0 days 00:03:00'), Timedelta('0 days 00:03:00')]


## Setting up the main timeseries and dataframes

In [54]:
# No duplicate date for each stations so the aggfunc doesn't matter a lot 

available_ts_df = pd.pivot_table(df, index='date', columns='station_name', values='available', aggfunc='first')
operative_ts_df = pd.pivot_table(df, index='date', columns='station_name', values='operative', aggfunc='first')

# Create a geodataframe for station proximity

info_df = df.groupby(by='station_name')['station_geo'].first().str.split(',', expand=True)
info_df.columns = ['lat', 'lon']
cap_df = df.groupby(by='station_name')['capacity'].max()

info_df = info_df.join(cap_df)

In [55]:
display(info_df)

,lat,lon,capacity
station_name,,,
11 Novembre 1918 - 8 Mai 1945,48.80890,2.53824,36
18 juin 1940 - Buzenval,48.86881,2.18543,25
8 Mai 1945 - 10 Juillet 1940,48.78457,2.39790,30
Abbeville - Faubourg Poissonnière,48.87922,2.34915,14
Abbé Carton - Plantes,48.82767,2.32092,25
...,...,...,...
Wurtz - Tolbiac,48.82631,2.34459,28
Youri Gagarine - Commune de Paris,48.78243,2.39397,33
Youri Gagarine - Karl Marx,48.78448,2.35908,22


In [15]:
display(df[df['station_name'] == 'Hôpital Européen Georges Pompidou'])
display(df[df['station_name'] == 'Jean Jaurès - Paul Lafargue'])
display(df)


,date,capacity,station_name,station_geo,operative,available


,date,capacity,station_name,station_geo,operative,available


,date,capacity,station_name,station_geo,operative,available
0,2020-11-26 12:59:00+00:00,35,Benjamin Godard - Victor Hugo,"48.86598,2.27572",True,9
1,2020-11-26 12:59:00+00:00,55,André Mazet - Saint-André des Arts,"48.85376,2.33910",True,27
2,2020-11-26 12:59:00+00:00,20,Charonne - Robert et Sonia Delauney,"48.85591,2.39257",True,0
3,2020-11-26 12:59:00+00:00,21,Toudouze - Clauzel,"48.87930,2.33736",True,1
4,2020-11-26 12:59:00+00:00,30,Mairie du 12ème,"48.84086,2.38755",True,4
...,...,...,...,...,...,...
10986725,2021-04-09 14:37:00+00:00,38,Général Michel Bizot - Claude Decaen,"48.83481,2.40093",True,6
10986726,2021-04-09 14:37:00+00:00,20,Ivry - Baudricourt,"48.82470,2.36311",True,3
10986727,2021-04-09 14:37:00+00:00,39,Saint-Mandé - Docteur Arnold Netter,"48.84463,2.40495",True,17
10986728,2021-04-09 14:37:00+00:00,21,Saint-Marcel - Hôpital,"48.83950,2.36099",True,16
